In [ ]:
!git clone https://github.com/TachibanaYoshino/AnimeGANv3.git


Cloning into 'AnimeGANv3'...
remote: Enumerating objects: 531, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 531 (delta 54), reused 29 (delta 24), pack-reused 461 (from 1)
Receiving objects: 100% (531/531), 359.63 MiB | 15.48 MiB/s, done.
Resolving deltas: 100% (118/118), done.
Updating files: 100% (216/216), done.


Interactive Demo for AnimeGANv2:FacePortraitv2 created by @xhlulu

Learn more about the model here: https://github.com/bryandlee/animegan2-pytorch

To start using this, run the cells with `Ctrl+F9` or "Runtime > Run All"

For accelerated inference, you can use a GPU. Simply select "Runtime > Change runtime type" and select "GPU" in the "Hardware Acceleration" dropdown.


In [ ]:
!cd /content/AnimeGANv3 && cd tools && python edge_smooth.py --dataset Hayao --img_size 256

Traceback (most recent call last):
  File "/content/AnimeGANv3/tools/edge_smooth.py", line 2, in <module>
    from tools.utils import check_folder
ModuleNotFoundError: No module named 'tools'


In [ ]:
!ls

AnimeGANv3  sample_data


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1' -O /content/AnimeGANv3_Hayao_STYLE_36.onnx


--2024-11-03 12:34:16--  https://docs.google.com/uc?export=download&id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1
Resolving docs.google.com (docs.google.com)... 173.194.79.113, 173.194.79.100, 173.194.79.102, ...
Connecting to docs.google.com (docs.google.com)|173.194.79.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1&export=download [following]
--2024-11-03 12:34:17--  https://drive.usercontent.google.com/download?id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.69.132, 2a00:1450:4013:c04::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.69.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4232991 (4.0M) [application/octet-stream]
Saving to: ‘/content/AnimeGANv3_Hayao_STYLE_36.onnx’

/content/AnimeGANv3 100%[===================

In [ ]:
!pip install onnxruntime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.9 MB/s eta 0:00:00


In [ ]:
from io import BytesIO
import cv2
import numpy as np
import torch
from PIL import Image
import ipywidgets as widgets
import IPython.display as display
from google.colab import files
import os
import onnxruntime as ort
import time

# Load the AnimeGANv3 model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = 'AnimeGANv3_Hayao_STYLE_36'  # Model name
session = ort.InferenceSession(f'/content/{model}.onnx', providers=['CUDAExecutionProvider' if device == 'cuda' else 'CPUExecutionProvider'])

# Load the face2paint model
face2paint_model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)

# Function to preprocess image for AnimeGANv3
def process_image(img):
    img = cv2.resize(img, (512, 512))  # Resize image to expected size
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 127.5 - 1.0
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to convert non-face areas using AnimeGANv3
def convert_non_face(img, original_shape):
    x = session.get_inputs()[0].name
    y = session.get_outputs()[0].name

    fake_img = session.run(None, {x: img})[0]

    # Postprocess output image
    images = (np.squeeze(fake_img) + 1.) / 2 * 255
    images = np.clip(images, 0, 255).astype(np.uint8)

    # Resize to original shape
    output_image = cv2.resize(images, (original_shape[1], original_shape[0]))  # Resize back to original
    return cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)

# Function to extract face using OpenCV's Haar Cascade
def extract_face(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img_array = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)  # Convert to BGR for OpenCV
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        face = img_array[y:y+h, x:x+w]
        return face, (x, y, w, h)  # Return the cropped face and its location
    return None, None  # Return None if no face is detected

# Function to process video
def process_video(input_video_path):
    cap = cv2.VideoCapture(input_video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    output_video_path = "output_video.mp4"

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (frame_width, frame_height))

    start_time = time.time()  # Start timer

    for frame_number in range(total_frames):
        ret, frame = cap.read()
        if not ret:
            break

        # Extract face from the frame
        face, coords = extract_face(frame)

        # If a face is detected
        if face is not None:
            # Transform face using face2paint
            face_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
            im_out = face2paint(face2paint_model, face_pil, side_by_side=False)

            # Convert the output to a NumPy array
            im_out_np = np.array(im_out)

            # Resize the transformed face to match the detected face's dimensions
            resized_face = cv2.resize(im_out_np, (coords[2], coords[3]))  # Resize to (width, height)

            # Replace the original face in the frame with the transformed face
            frame[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]] = cv2.cvtColor(resized_face, cv2.COLOR_RGB2BGR)

            # Transform non-face areas using AnimeGANv3
            full_transformed_img = convert_non_face(process_image(frame), frame.shape)  # Keep original shape

            # Create a mask and combine images
            mask = np.ones(frame.shape[:2], dtype=np.uint8) * 255
            mask[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]] = 0
            frame[mask == 255] = full_transformed_img[mask == 255]

        out.write(frame)

        # Calculate elapsed time and estimate remaining time
        elapsed_time = time.time() - start_time
        remaining_time = (elapsed_time / (frame_number + 1)) * (total_frames - (frame_number + 1))
        print(f"Processed frame {frame_number + 1}/{total_frames}. Estimated time remaining: {remaining_time:.2f} seconds.")

    cap.release()
    out.release()
    return output_video_path

# User interface
button = widgets.Button(description="Upload Video to Transform")
output = widgets.Output()

def run(b):
    button.disabled = True

    with output:
        display.clear_output()

    uploaded = files.upload()

    for fname in uploaded:
        input_video_path = fname
        output_video_path = process_video(input_video_path)
        print(f"Transformed video saved at: {output_video_path}")

    button.disabled = False

button.on_click(run)
display.display(button, output)


Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main
Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main


Button(description='Upload Video to Transform', style=ButtonStyle())

Output()

Saving Video2_AnimeGANv3.mp4 to Video2_AnimeGANv3 (3).mp4
Processed frame 1/565. Estimated time remaining: 5321.22 seconds.
Processed frame 2/565. Estimated time remaining: 4914.35 seconds.
Processed frame 3/565. Estimated time remaining: 4982.97 seconds.
Processed frame 4/565. Estimated time remaining: 4873.49 seconds.
Processed frame 5/565. Estimated time remaining: 4823.89 seconds.
Processed frame 6/565. Estimated time remaining: 4837.31 seconds.
Processed frame 7/565. Estimated time remaining: 4742.57 seconds.
Processed frame 8/565. Estimated time remaining: 4764.40 seconds.
Processed frame 9/565. Estimated time remaining: 4759.49 seconds.
Processed frame 10/565. Estimated time remaining: 4706.42 seconds.
Processed frame 11/565. Estimated time remaining: 4723.24 seconds.
Processed frame 12/565. Estimated time remaining: 4683.55 seconds.
Processed frame 13/565. Estimated time remaining: 4679.39 seconds.
Processed frame 14/565. Estimated time remaining: 4688.09 seconds.
Processed fra

In [3]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1' -O /content/AnimeGANv3_Hayao_STYLE_36.onnx
!pip install onnxruntime
!pip install gradio

--2024-11-08 12:57:13--  https://docs.google.com/uc?export=download&id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1
Resolving docs.google.com (docs.google.com)... 142.251.2.102, 142.251.2.100, 142.251.2.138, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1&export=download [following]
--2024-11-08 12:57:13--  https://drive.usercontent.google.com/download?id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.137.132, 2607:f8b0:4023:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.137.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4232991 (4.0M) [application/octet-stream]
Saving to: ‘/content/AnimeGANv3_Hayao_STYLE_36.onnx’

/content/AnimeGANv3 100%[===================>]  

In [4]:
import gradio as gr
import cv2
import numpy as np

# Function to process the video and generate anime style output
def process_video(input_video):
    cap = cv2.VideoCapture(input_video)
    anime_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Apply your anime transformation here
        anime_frame = apply_anime_style(frame)  # Replace with your actual function
        anime_frames.append(anime_frame)

    cap.release()

    output_video = "output_anime_video.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, 30.0, (anime_frames[0].shape[1], anime_frames[0].shape[0]))

    for frame in anime_frames:
        out.write(frame)

    out.release()
    return output_video

# Function to evaluate the animeness of the video
def evaluate_animeness(video_path):
    cap = cv2.VideoCapture(video_path)
    score = 0
    num_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        num_frames += 1
        score += analyze_frame_for_animeness(frame)  # Replace with your actual analysis function

    cap.release()

    if num_frames > 0:
        score /= num_frames

    return score

# Gradio interface function
def generate_anime_video(input_video):
    anime_video = process_video(input_video)
    animeness_score = evaluate_animeness(input_video)
    return anime_video, f"Animeness Score: {animeness_score:.2f}"

# Gradio interface
interface = gr.Interface(
    fn=generate_anime_video,
    inputs=gr.Video(label="Input Video"),
    outputs=[gr.Video(label="Anime Video"), gr.Textbox(label="Animeness Score")],
    title="Anime Video Generation with Evaluation",
    description="Convert a normal video into anime style and evaluate its animeness."
)

# Launch the Gradio interface
interface.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3ca30adff9c14d430c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [17]:
# !wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1' -O /kaggle/working/AnimeGANv3_Hayao_STYLE_36.onnx
!pip install gradio
!pip install onnxruntime
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1' -O /content/AnimeGANv3_Hayao_STYLE_36.onnx


--2024-11-08 14:10:35--  https://docs.google.com/uc?export=download&id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1
Resolving docs.google.com (docs.google.com)... 142.251.2.113, 142.251.2.138, 142.251.2.139, ...
Connecting to docs.google.com (docs.google.com)|142.251.2.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1&export=download [following]
--2024-11-08 14:10:35--  https://drive.usercontent.google.com/download?id=1ud-QxEtdp4bMODFUMHCZluy2Ic0Yfns1&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.251.2.132, 2607:f8b0:4023:c03::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.251.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4232991 (4.0M) [application/octet-stream]
Saving to: ‘/content/AnimeGANv3_Hayao_STYLE_36.onnx’

/content/AnimeGANv3 100%[===================>]   4

/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main
Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1adfc208d79e8a0d1a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Need 'write' access token to create a Spaces repo.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

Enter your token (input will not be visible): 

In [16]:
import cv2
import numpy as np
import torch
import onnxruntime as ort
from PIL import Image
import gradio as gr
import json
from skimage.feature import graycomatrix, graycoprops

# Load the AnimeGANv3 model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = 'AnimeGANv3_Hayao_STYLE_36'  # Model name
session = ort.InferenceSession(f'/content/{model}.onnx', providers=['CUDAExecutionProvider' if device == 'cuda' else 'CPUExecutionProvider'])

# Load the face2paint model
face2paint_model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)

# Function to preprocess image for AnimeGANv3
def process_image(img):
    img = cv2.resize(img, (512, 512))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 127.5 - 1.0
    img = np.expand_dims(img, axis=0)
    return img

# Function to convert non-face areas using AnimeGANv3
def convert_non_face(img, original_shape):
    x = session.get_inputs()[0].name
    fake_img = session.run(None, {x: img})[0]
    images = (np.squeeze(fake_img) + 1.) / 2 * 255
    images = np.clip(images, 0, 255).astype(np.uint8)
    output_image = cv2.resize(images, (original_shape[1], original_shape[0]))
    return cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)

# Function to extract face using OpenCV's Haar Cascade
def extract_face(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img_array = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        face = img_array[y:y+h, x:x+w]
        return face, (x, y, w, h)
    return None, None

# Function to calculate edge density
def calculate_edge_density(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 100, 200)
    return np.sum(edges) / (image.shape[0] * image.shape[1])

# Function to calculate color metrics
def calculate_color_metrics(image):
    mean_color = np.mean(image, axis=(0, 1))
    std_dev_color = np.std(image, axis=(0, 1))
    return mean_color, std_dev_color

# Function to calculate texture metrics using GLCM
def calculate_texture_metrics(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray_image, distances=[1], angles=[0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    total_variation = np.sum(np.abs(np.diff(gray_image, axis=0))) + np.sum(np.abs(np.diff(gray_image, axis=1)))

    return contrast, correlation, energy, homogeneity, total_variation

# Function to process video and analyze metrics
def process_and_analyze_video(video_file):
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Video Writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('/content/output_video.avi', fourcc, fps, (width, height))

    # Variables to store metrics
    frame_metrics = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        face, coords = extract_face(frame)

        if face is not None:
            face_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
            im_out = face2paint(face2paint_model, face_pil, side_by_side=False)
            im_out_np = np.array(im_out)
            resized_face = cv2.resize(im_out_np, (coords[2], coords[3]))
            frame[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]] = resized_face

        full_transformed_img = convert_non_face(process_image(frame), frame.shape)

        # Mask for non-face areas
        mask = np.ones(frame.shape[:2], dtype=np.uint8) * 255
        if face is not None:
            mask[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]] = 0
        frame[mask == 255] = full_transformed_img[mask == 255]

        # Collect metrics for animeness score
        edge_density = calculate_edge_density(frame)
        mean_color, std_dev_color = calculate_color_metrics(frame)
        contrast, correlation, energy, homogeneity, total_variation = calculate_texture_metrics(frame)

        frame_metrics.append({
            'Mean Color': mean_color,
            'Standard Deviation of Color': std_dev_color,
            'Edge Density': edge_density,
            'Total Variation': total_variation,
            'Contrast': contrast,
            'Correlation': correlation,
            'Energy': energy,
            'Homogeneity': homogeneity
        })

        # Write processed frame to output
        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    # Average metrics
    metrics_average = {
        'Mean Color': np.mean([metric['Mean Color'] for metric in frame_metrics], axis=0),
        'Standard Deviation of Color': np.mean([metric['Standard Deviation of Color'] for metric in frame_metrics], axis=0),
        'Edge Density': np.mean([metric['Edge Density'] for metric in frame_metrics]),
        'Total Variation': np.mean([metric['Total Variation'] for metric in frame_metrics]),
        'Contrast': np.mean([metric['Contrast'] for metric in frame_metrics]),
        'Correlation': np.mean([metric['Correlation'] for metric in frame_metrics]),
        'Energy': np.mean([metric['Energy'] for metric in frame_metrics]),
        'Homogeneity': np.mean([metric['Homogeneity'] for metric in frame_metrics])
    }

    return '/content/output_video.avi', metrics_average

# Function to load histogram data from JSON file
def load_histogram_data(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    return data

# Function to calculate animeness score
def calculate_anime_metrics(video_metrics, folder_metrics):
    # Here you can define weights for each metric based on their importance
    weights = {
        'Mean Color': 0.15,
        'Standard Deviation of Color': 0.15,
        'Edge Density': 0.1,
        'Total Variation': 0.1,
        'Contrast': 0.15,
        'Correlation': 0.1,
        'Energy': 0.15,
        'Homogeneity': 0.1
    }

    # Calculate the animeness score
    animeness_score = 0
    for metric in weights.keys():
        # Normalize metrics from video and folder metrics to the same scale (0-1)
        video_value = video_metrics[metric]
        folder_value = folder_metrics.get(metric, np.mean(video_metrics[metric]))  # Default to mean if not found
        animeness_score += weights[metric] * ((video_value + folder_value) / 2)

    return animeness_score

# Gradio Interface
def gradio_interface(video):
    # Process the input video and get the transformed output and metrics
    transformed_video, video_metrics = process_and_analyze_video(video)

    # Load original metrics from JSON
    json_file = 'mean_metrics.json'  # Replace with your actual histogram JSON file
    original_metrics = load_histogram_data(json_file)

    # Calculate animeness score
    animeness_score = calculate_anime_metrics(video_metrics, original_metrics)

    return video, transformed_video, {'Video Metrics': video_metrics, 'Original Metrics': original_metrics, 'Animeness Score': animeness_score}

# Launch Gradio app
gr.Interface(
    fn=gradio_interface,
    inputs='file',
    outputs=[gr.Video(label="Original Video"), gr.Video(label="Transformed Video"), gr.JSON(label="Metrics")],
    title="Anime Video Transformation",
    description="Upload a video to transform it into an anime style and analyze its metrics."
).launch()


/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main
Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://288f6acae236c84879.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
import cv2
import numpy as np
import torch
import onnxruntime as ort
from PIL import Image
import gradio as gr
import json
from skimage.feature import graycomatrix, graycoprops
import os
import math
# Load the AnimeGANv3 model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = 'AnimeGANv3_Hayao_STYLE_36'
session = ort.InferenceSession(f'/content/{model}.onnx', providers=['CUDAExecutionProvider' if device == 'cuda' else 'CPUExecutionProvider'])

# Load the face2paint model
face2paint_model = torch.hub.load("bryandlee/animegan2-pytorch:main", "generator", device=device).eval()
face2paint = torch.hub.load("bryandlee/animegan2-pytorch:main", "face2paint", device=device)

# Function to preprocess image for AnimeGANv3
def process_image(img):
    img = cv2.resize(img, (512, 512))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float32) / 127.5 - 1.0
    img = np.expand_dims(img, axis=0)
    return img

# Function to convert non-face areas using AnimeGANv3
def convert_non_face(img, original_shape):
    x = session.get_inputs()[0].name
    fake_img = session.run(None, {x: img})[0]
    images = (np.squeeze(fake_img) + 1.) / 2 * 255
    images = np.clip(images, 0, 255).astype(np.uint8)
    output_image = cv2.resize(images, (original_shape[1], original_shape[0]))
    return cv2.cvtColor(output_image, cv2.COLOR_RGB2BGR)

# Function to extract face using OpenCV's Haar Cascade
def extract_face(image):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    img_array = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        face = img_array[y:y+h, x:x+w]
        return face, (x, y, w, h)
    return None, None

# Function to calculate edge density
def calculate_edge_density(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray_image, 100, 200)
    return np.sum(edges) / (image.shape[0] * image.shape[1])

# Function to calculate color metrics
def calculate_color_metrics(image):
    mean_color = np.mean(image, axis=(0, 1))
    std_dev_color = np.std(image, axis=(0, 1))
    return mean_color, std_dev_color

# Function to calculate texture metrics using GLCM
def calculate_texture_metrics(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    glcm = graycomatrix(gray_image, distances=[1], angles=[0], symmetric=True, normed=True)
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]
    total_variation = np.sum(np.abs(np.diff(gray_image, axis=0))) + np.sum(np.abs(np.diff(gray_image, axis=1)))

    return contrast, correlation, energy, homogeneity, total_variation

# Process and analyze video
def process_and_analyze_video(video_file):
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out_path = '/content/output_video.avi'
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

    frame_metrics = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        face, coords = extract_face(frame)

        if face is not None:
            face_pil = Image.fromarray(cv2.cvtColor(face, cv2.COLOR_BGR2RGB))
            im_out = face2paint(face2paint_model, face_pil, side_by_side=False)
            im_out_np = np.array(im_out)
            resized_face = cv2.resize(im_out_np, (coords[2], coords[3]))
            frame[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]] = resized_face

        full_transformed_img = convert_non_face(process_image(frame), frame.shape)

        mask = np.ones(frame.shape[:2], dtype=np.uint8) * 255
        if face is not None:
            mask[coords[1]:coords[1]+coords[3], coords[0]:coords[0]+coords[2]] = 0
        frame[mask == 255] = full_transformed_img[mask == 255]

        edge_density = calculate_edge_density(frame)
        mean_color, std_dev_color = calculate_color_metrics(frame)
        contrast, correlation, energy, homogeneity, total_variation = calculate_texture_metrics(frame)

        frame_metrics.append({
            'Mean Color': mean_color,
            'Standard Deviation of Color': std_dev_color,
            'Edge Density': edge_density,
            'Total Variation': total_variation,
            'Contrast': contrast,
            'Correlation': correlation,
            'Energy': energy,
            'Homogeneity': homogeneity
        })

        out.write(frame)

    cap.release()
    out.release()
    cv2.destroyAllWindows()

    metrics_average = {
        'Mean Color': np.mean([metric['Mean Color'] for metric in frame_metrics], axis=0),
        'Standard Deviation of Color': np.mean([metric['Standard Deviation of Color'] for metric in frame_metrics], axis=0),
        'Edge Density': np.mean([metric['Edge Density'] for metric in frame_metrics]),
        'Total Variation': np.mean([metric['Total Variation'] for metric in frame_metrics]),
        'Contrast': np.mean([metric['Contrast'] for metric in frame_metrics]),
        'Correlation': np.mean([metric['Correlation'] for metric in frame_metrics]),
        'Energy': np.mean([metric['Energy'] for metric in frame_metrics]),
        'Homogeneity': np.mean([metric['Homogeneity'] for metric in frame_metrics])
    }

    return out_path, metrics_average

def load_histogram_data(json_file):
    with open(json_file, 'r') as file:
        data = json.load(file)
    return data

def calculate_anime_metrics(video_metrics, folder_metrics):
    weights = {
        'Mean Color': 0,
        'Standard Deviation of Color': 0,
        'Edge Density': 0.25,
        'Total Variation': 0.0,
        'Contrast': 0.15,
        'Correlation': 0.2,
        'Energy': 0.2,
        'Homogeneity': 0.2
    }

    animeness_score = 0
    for metric in weights.keys():
        video_value = video_metrics[metric]
        folder_value = folder_metrics.get(metric, np.mean(video_metrics[metric]))
        animeness_score += weights[metric] * (abs((video_value - folder_value)))

    return abs(2/(1+math.exp(-0.01*animeness_score[0])) - 2)

def gradio_interface(video):
    transformed_video, video_metrics = process_and_analyze_video(video)
    json_file = 'mean_metrics.json'
    original_metrics = load_histogram_data(json_file)
    animeness_score = calculate_anime_metrics(video_metrics, original_metrics)

    return video, transformed_video, {'Video Metrics': video_metrics, 'Original Metrics': original_metrics, 'Animeness Score': animeness_score}

gr.Interface(
    fn=gradio_interface,
    inputs='file',
    outputs=[gr.Video(label="Original Video"), gr.Video(label="Transformed Video"), gr.JSON(label="Metrics")],
    title="Anime Video Transformation",
    description="Upload a video to transform it into an anime style and analyze its metrics."
).launch(debug=True)


/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main
Using cache found in /root/.cache/torch/hub/bryandlee_animegan2-pytorch_main


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://8fc2c1057dae0568aa.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/gradio/components/video.py:337: UserWarning: Video does not have browser-compatible container or codec. Converting to mp4.
  warnings.warn(
